In [1]:
import pandas as pd
import numpy as np


In [6]:
x_arr = np.linspace(0,100)
df = pd.DataFrame({"x":x_arr,"y":[np.sin(x) for x in x_arr]})
# df.to_csv("../../test_files/test_file_sin.csv",index=False)
# df.to_excel("../../test_files/test_file_sin.xlsx",index=False)

df.head()

,x,y
0,0.000000,0.000000
1,2.040816,0.891559
2,4.081633,-0.807582
3,6.122449,-0.160045
4,8.163265,0.952552


In [5]:
# with open("tmp.txt","w") as f:
#     f.write(str(df))

In [11]:
import os

def get_extension_from_filename(filename:str):
    return os.path.splitext(filename)[-1][1:]

In [12]:
get_extension_from_filename("tmp.csv.txt")

'txt'

In [10]:
from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model='codellama', messages=[{
    'role': 'system',
    'content': 'You can only write back in German.',
  },
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])
# or access fields directly from the response object
print(response.message.content)

Die Sonne strahlt mit seiner Energie in alle Richtungen aus, aber die meiste davon wird zurückgeworfen nach oben. Dieses Strahlungsmuster erklärt, warum der Himmel blau aussieht.
Die Sonne strahlt mit seiner Energie in alle Richtungen aus, aber die meiste davon wird zurückgeworfen nach oben. Dieses Strahlungsmuster erklärt, warum der Himmel blau aussieht.


In [15]:
import yaml
with open("../../config/config.yaml") as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)

In [17]:
cfg

{'coding_model_name': 'codellama',
 'coding_model_instruction': 'You are an Large Language Model that generates python code to solve plotting and visualization problems, writing only in Python, and you only return valid code. No explanation just the pure runable code.'}

In [ ]:
from models.plotting_agent import ChatWithOllama

In [4]:
chatbot = ChatWithOllama()
response = chatbot.call_LLM(prompt="Write me a function that return all prime numbers under a number")

In [32]:
response[0] == "`"

True

In [33]:
response

'```python\ndef get_prime_numbers(n):\n    prime_numbers = []\n    for i in range(2, n+1):\n        if i > 1:\n            for j in range(2, i):\n                if i % j == 0:\n                    break\n            else:\n                prime_numbers.append(i)\n    return prime_numbers\n```'

In [40]:
re.search(r"```python([\s\S]*?)\n```",response).group(1)

'\ndef get_prime_numbers(n):\n    prime_numbers = []\n    for i in range(2, n+1):\n        if i > 1:\n            for j in range(2, i):\n                if i % j == 0:\n                    break\n            else:\n                prime_numbers.append(i)\n    return prime_numbers'